Imagine that we have a map of the united states, but instead of having individual states, we do pie charts --> and the pie charts are for the generation mix, and we have whatever size wedge depending on coal nuclear wind, and we make the size of the pie linked to the quantity of electricity that they generate in the state
and it's important that we use generation
- Use a marimekko chart to identify the differences

Have arrows that go from one pie chart to another to see how one pie chart to another --
Hawaii and Alaska off to the side without any arrows, not all states are going to have arrows that go to the other 47
- 
plotly height

Only focusing on electricity that's getting transmitted along state lines

Just take the most recent year that they have
Query the SEDS API

Have 3 charts
- One map is how much electricty does each state produce
- What is the generation mix per state
- How much is going in and out

Don't have each map representing of the state, have boxes distributed into a map, bc obviously its not gonna be the easiest thing to put a number
- Map might not be the best idea, could also 

Can also have dark red if they're bringing in a lot of electricity, and dark blue if they're sending out a lot
Do we want to compare absolute quantities that they send in and out or what share of total generation
- i.e. California possesses the largest share, but do they possess the largest absolute amount

Quantity generated --> you have a number but you have 5 different shades of different thresholds that look like appropriate thresholds when you eyeball the data --> heatmap

Here we have a list of columns that we'll be using to map out interestate electricity transfers. I selected every fuel type, but because net generation is only recorded for non-fossil fuel energy source,s we'll have to convert how much electricity is generated using the conversion factor, which is likely the efficiency from thermal power generation.

In [75]:
pd.DataFrame({'Chosen Data':df.seriesDescription.unique()}).style

,Chosen Data
0,Net interstate flow of electricity (negative indicates flow out of state)
1,Nuclear electricity net generation in the electric power sector
2,Net interstate flow of electricity and associated losses (negative indicates flow out of state)
3,Hydroelectricity net generation in the electric power sector
4,Factor for converting electricity generated from nuclear power from physical units to Btu
5,Distillate fuel oil consumed by the electric power sector
6,Geothermal electricity net generation in the electric power sector
7,Solar thermal and photovoltaic electricity net generation in the electric power sector
8,Wind electricity net generation in the electric power sector
9,Factor for converting coal consumed by the electric power sector from physical units to Btu


In [81]:
import requests
import pandas as pd
import geopandas as gpd

In [22]:
key = 'FsDFoQS21gEmrDViRY9RpJqYtmJ5YizlncdbyKaE'

In [29]:
api_link_2 = 'https://api.eia.gov/v2/seds/data/?api_key='+key+'&frequency=annual&data[0]=value&facets[stateId][]=AK&facets[stateId][]=AL&facets[stateId][]=AR&facets[stateId][]=AZ&facets[stateId][]=CA&facets[stateId][]=CO&facets[stateId][]=CT&facets[stateId][]=DC&facets[stateId][]=DE&facets[stateId][]=FL&facets[stateId][]=GA&facets[stateId][]=HI&facets[stateId][]=IA&facets[stateId][]=ID&facets[stateId][]=IL&facets[stateId][]=IN&facets[stateId][]=KS&facets[stateId][]=KY&facets[stateId][]=LA&facets[stateId][]=MA&facets[stateId][]=MD&facets[stateId][]=ME&facets[stateId][]=MI&facets[stateId][]=MN&facets[stateId][]=MO&facets[stateId][]=MS&facets[stateId][]=MT&facets[stateId][]=NC&facets[stateId][]=ND&facets[stateId][]=NE&facets[stateId][]=NH&facets[stateId][]=NJ&facets[stateId][]=NM&facets[stateId][]=NV&facets[stateId][]=NY&facets[stateId][]=OH&facets[stateId][]=OK&facets[stateId][]=OR&facets[stateId][]=PA&facets[stateId][]=RI&facets[stateId][]=SC&facets[stateId][]=SD&facets[stateId][]=TN&facets[stateId][]=TX&facets[stateId][]=US&facets[stateId][]=UT&facets[stateId][]=VA&facets[stateId][]=VT&facets[stateId][]=WA&facets[stateId][]=WI&facets[stateId][]=WV&facets[stateId][]=WY&facets[stateId][]=X3&facets[stateId][]=X5&facets[seriesId][]=CLEIB&facets[seriesId][]=CLEIK&facets[seriesId][]=DFEIB&facets[seriesId][]=DFTCK&facets[seriesId][]=ELISB&facets[seriesId][]=ELISP&facets[seriesId][]=GEEGP&facets[seriesId][]=HYEGP&facets[seriesId][]=NGEIB&facets[seriesId][]=NGEIK&facets[seriesId][]=NUEGP&facets[seriesId][]=NUETK&facets[seriesId][]=PAEIB&facets[seriesId][]=PAEIK&facets[seriesId][]=SOEGP&facets[seriesId][]=WYEGP&start=2020&end=2021&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000'
api_link = 'https://api.eia.gov/v2/seds/data/seriesid=WYTXP?api_key='+key+'&frequency=annual&data[0]=value&start=2020&end=2021&sort[0][column]=seriesId&sort[0][direction]=desc&offset=0&length=5000'
r = requests.get(api_link_2)
d = r.json()

In [30]:
print(d)

{'response': {'total': 679, 'dateFormat': 'YYYY', 'frequency': 'annual', 'data': [{'period': 2021, 'seriesId': 'ELISP', 'seriesDescription': 'Net interstate flow of electricity (negative indicates flow out of state)', 'stateId': 'TN', 'stateDescription': 'Tennessee', 'value': 28451, 'unit': 'Million kilowatthours'}, {'period': 2021, 'seriesId': 'NUEGP', 'seriesDescription': 'Nuclear electricity net generation in the electric power sector', 'stateId': 'IN', 'stateDescription': 'Indiana', 'value': 0, 'unit': 'Million kilowatthours'}, {'period': 2021, 'seriesId': 'NUEGP', 'seriesDescription': 'Nuclear electricity net generation in the electric power sector', 'stateId': 'KS', 'stateDescription': 'Kansas', 'value': 8575, 'unit': 'Million kilowatthours'}, {'period': 2021, 'seriesId': 'NUEGP', 'seriesDescription': 'Nuclear electricity net generation in the electric power sector', 'stateId': 'ME', 'stateDescription': 'Maine', 'value': 0, 'unit': 'Million kilowatthours'}, {'period': 2021, 'seri

In [42]:
d['response'] #exploring the data, see how we can wrap this in a dataframe

{'total': 679,
 'dateFormat': 'YYYY',
 'frequency': 'annual',
 'data': [{'period': 2021,
   'seriesId': 'ELISP',
   'seriesDescription': 'Net interstate flow of electricity (negative indicates flow out of state)',
   'stateId': 'TN',
   'stateDescription': 'Tennessee',
   'value': 28451,
   'unit': 'Million kilowatthours'},
  {'period': 2021,
   'seriesId': 'NUEGP',
   'seriesDescription': 'Nuclear electricity net generation in the electric power sector',
   'stateId': 'IN',
   'stateDescription': 'Indiana',
   'value': 0,
   'unit': 'Million kilowatthours'},
  {'period': 2021,
   'seriesId': 'NUEGP',
   'seriesDescription': 'Nuclear electricity net generation in the electric power sector',
   'stateId': 'KS',
   'stateDescription': 'Kansas',
   'value': 8575,
   'unit': 'Million kilowatthours'},
  {'period': 2021,
   'seriesId': 'NUEGP',
   'seriesDescription': 'Nuclear electricity net generation in the electric power sector',
   'stateId': 'ME',
   'stateDescription': 'Maine',
   'va

In [62]:
df = pd.DataFrame(d['response']['data'])

In [63]:
df.head()

,period,seriesId,seriesDescription,stateId,stateDescription,value,unit
0,2021,ELISP,Net interstate flow of electricity (negative i...,TN,Tennessee,28451.0,Million kilowatthours
1,2021,NUEGP,Nuclear electricity net generation in the elec...,IN,Indiana,0.0,Million kilowatthours
2,2021,NUEGP,Nuclear electricity net generation in the elec...,KS,Kansas,8575.0,Million kilowatthours
3,2021,NUEGP,Nuclear electricity net generation in the elec...,ME,Maine,0.0,Million kilowatthours
4,2021,ELISB,Net interstate flow of electricity and associa...,NJ,New Jersey,159470.0,Billion Btu


In [93]:
df.isnull().sum()/df.shape[0] #checking for missing values

period               0.0
seriesId             0.0
seriesDescription    0.0
stateId              0.0
stateDescription     0.0
value                0.0
unit                 0.0
dtype: float64

In [82]:
regions = gpd.read_file('cb_2018_us_region_500k.shp')

In [83]:
regions.head()

,REGIONCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,1,0200000US1,1,Northeast,68,419357835545,50259300137,"MULTIPOLYGON (((-68.27472 44.25867, -68.27144 ..."
1,2,0200000US2,2,Midwest,68,1943997274253,184273267512,"MULTIPOLYGON (((-82.73571 41.60336, -82.73392 ..."
2,4,0200000US4,4,West,68,4536201747682,316587292459,"MULTIPOLYGON (((179.48246 51.98283, 179.48656 ..."
3,3,0200000US3,3,South,68,2249871668369,134084610547,"MULTIPOLYGON (((-75.56555 39.51485, -75.56174 ..."


In [84]:
census = pd.read_csv('censusregions.csv')

In [85]:
census.head()

,State,State Code,Region,Division
0,Alaska,AK,West,Pacific
1,Alabama,AL,South,East South Central
2,Arkansas,AR,South,West South Central
3,Arizona,AZ,West,Mountain
4,California,CA,West,Pacific


In [88]:
census.Region.unique()

array(['West', 'South', 'Northeast', 'Midwest'], dtype=object)

In [91]:
census_region = census.merge(regions, how = 'left', left_on = 'Region',right_on = 'NAME')
census_region.drop(columns = 'NAME',inplace=True)

In [92]:
census_region.head()

,State,State Code,Region,Division,REGIONCE,AFFGEOID,GEOID,LSAD,ALAND,AWATER,geometry
0,Alaska,AK,West,Pacific,4,0200000US4,4,68,4536201747682,316587292459,"MULTIPOLYGON (((179.48246 51.98283, 179.48656 ..."
1,Alabama,AL,South,East South Central,3,0200000US3,3,68,2249871668369,134084610547,"MULTIPOLYGON (((-75.56555 39.51485, -75.56174 ..."
2,Arkansas,AR,South,West South Central,3,0200000US3,3,68,2249871668369,134084610547,"MULTIPOLYGON (((-75.56555 39.51485, -75.56174 ..."
3,Arizona,AZ,West,Mountain,4,0200000US4,4,68,4536201747682,316587292459,"MULTIPOLYGON (((179.48246 51.98283, 179.48656 ..."
4,California,CA,West,Pacific,4,0200000US4,4,68,4536201747682,316587292459,"MULTIPOLYGON (((179.48246 51.98283, 179.48656 ..."
